# Academic Success
## 2. Modélisation
- Ce notebook fait suite au notebook **01_EDA_Academic_Success** https://www.kaggle.com/code/mohamedabder/01-eda-academic-success

In [ ]:
from termcolor import colored

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import label_binarize

import time 


#modeles :
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

from sklearn import metrics


import pickle 
import warnings
import os

warnings.filterwarnings('ignore')

# Préparation des données
- Séparation train test set
- Encodage de la target et enregistrement de l'encoder

In [ ]:
path1 = 'D:\\etude_data_science\\kaggle_competition\\07_academic_success\\dataset\\'
os.listdir(path1)

In [ ]:
df_preprocessed_path = os.path.join(path1,os.listdir(path1)[0])

In [ ]:
df = pd.read_csv(df_preprocessed_path, index_col = 0)
df.head()

In [ ]:
path_scaler = os.path.join(os.getcwd(),"scaler.pkl")
path_labelencoder = os.path.join(os.getcwd(),"target_encoder.pkl")

In [ ]:
# Ouvrir le fichier en mode binaire de lecture
with open(path_scaler, 'rb') as file:
    # Lire et désérialiser l'objet
    scaler = pickle.load(file)
print(f"Données chargées depuis {colored(path_scaler,'blue')} : {colored(scaler,'green',attrs=['bold'])}")

# Ouvrir le fichier en mode binaire de lecture
with open(path_labelencoder, 'rb') as file:
    # Lire et désérialiser l'objet
    labelencoder = pickle.load(file)
print(f"Données chargées depuis {colored(path_labelencoder,'blue')} : {colored(labelencoder,'green',attrs=['bold'])}")

In [ ]:
#Chargement du dataframe preprocessed : 
data_cleaned = pd.read_csv(path_train, index_col = 0)
df = data_cleaned.copy()

In [ ]:
#Récupération des types de variables
var_cont = list(df.select_dtypes("float"))
var_dis = list(df.select_dtypes("int"))
var_qual = list(df.select_dtypes('object'))

In [ ]:
df.shape

#### Séparation de l'ID et préparation du train et test set

In [ ]:
identifiant = df["id"]
df.drop(identifiant.name, axis = 1,inplace=True)

In [ ]:
X,y = df.drop("Target", axis =1), df["Target"]
X_train,X_test,y_train,y_test = train_test_split(X,y, stratify =y,  #pour rappel, dans la partie EDA nous avions remarqué que les classes n'étaient pas équilibrées
                                                 test_size=0.2, random_state = 42)

In [ ]:
print("Dimensions après split des données :\n")
print(f"X train :{X_train.shape}")
print(f"y train :{y_train.shape}")
print()
print(f"X test :{X_test.shape}")
print(f"y test :{y_test.shape}")

### Encodage de la target et enregistrement de l'encoder

In [ ]:
#Entrainement de l'encoder
labelencoder = LabelEncoder()
labelencoder.fit(y_train)

In [ ]:
#Enregistrement de l'encoder pré-entrainé
with open('target_encoder.pkl', 'wb') as f:
    pickle.dump(labelencoder, f)

In [ ]:
#Encodage de la target :
y_train,y_test = labelencoder.transform(y_train),labelencoder.transform(y_test)

- Maintenant que les données ont été préparées, nous pouvons désormais passer aux différentes étapes de la modélisation.

# Plan : 
#### 1. Création des fonctions de modélisation 

#### 2. Entrainement et enregistrement des modèles

#### 3. Résultats et premières selection de modèles

#### 4. Visualisation des autres metrics

#### 5. Amélioration des modèles par GridSearchCV

#### 6. Visualisation des performances

#  1. Création des fonctions de modélisation
### Fonction pour l'entrainement et la récupération des metrics des modèles :

In [ ]:
def train(model):
    """Fonction qui entraine un modèle et affiche le score obtenu sur 5 splits après
    cross validation et le score moyen. 
    Renvoi un tuple de trois éléments :
    (predictions, temps d'execution, le modèle entrainé)"""
    
    scores = cross_val_score(estimator=model, X=X_train.values, y=y_train)
    
    start_time = time.time()
    
    OneVsRestClassifier(model.fit(X_train,y_train)) #transforme un problème de classification
    #multi-classes en plusieurs problèmes de classification binaire.
    
    end_time = time.time()
    
    # durée totale de l'entrainement 
    training_time = round(end_time - start_time,2)
    
    # Durée en minute :
    training_time_min = round(training_time/60,2)

    yp = model.predict(X_test.values)
    
    print(colored(type(model).__name__,"light_green",attrs=["bold"]))
    print("Scores sur 5 splits : ", [i.round(3) for i in scores])
    print("Score moyen :", scores.mean().round(3))

    return (yp, training_time_min, model)

def metric(prediction, model_name):
    """Fonction qui renvoie un objet pandas Series contenant les valeurs des metrics
    principales : f1 score, accuracy, recall, precision score à partir de la liste des prédictions d'un modèle"""
    
    f1 = metrics.f1_score(y_true = y_test, y_pred = prediction, average='weighted')
    accuracy = metrics.accuracy_score(y_true = y_test, y_pred = prediction)
    recal = metrics.recall_score(y_true = y_test, y_pred = prediction, average='weighted')
    precision = metrics.precision_score(y_true = y_test, y_pred = prediction, average='weighted')
    
    all_metric = [f1,accuracy,recal,precision]
    
    index_metric = ["F1-score","Accuracy","Recall","Precision"]
    
    series = pd.Series(all_metric, name=str(model_name), index=index_metric)
    return pd.DataFrame(series)

def trainig_series(list_models):
    """Fonction qui entraine en série plusieurs modèle à la fois
    Renvoie un tuple contenant 4 éléments :
    DataFrame contenant toutes les métrics
    DataFrame contenant toutes les prédictions de modèles
    Dictionnaire contenant tout les modèles préentrainés
    Dictionnaire contenant le temps d'entrainement de chaque modèle
    """
    #Dictionnaire qui contiendra les différentes metrics de chaque modèle
    dict_metric = {}
    #dictionnaire contenant les valeurs prédictives pour chaque modèles
    dict_prediction = {}
    #dictionnaire contenant les modele entrainé
    dict_model_trained = {}
    #dictionnaire contenant le temps d'exécution 
    dict_time ={}
        
    
    for current_model in list_models:
        #Recuperation des prédictions et du temps d'execution de chaque modèles
        prediction, temps_exe, model_trained = train(current_model)

        # Récupération du nom du modèle et du chemin d'enregistrement
        model_name = type(current_model).__name__

        # Enregitrement des modèles après entrainement : 
        dict_model_trained[model_name] = model_trained

        #Enregistrement des prédiction dans le dictionnaire :
        dict_prediction[model_name] = prediction

        # Enregistrement des métriques au dictionnaire
        dict_metric[model_name] = metric(prediction=prediction, model_name=model_name)
        
        #Enregistrement du tps d'execution
        dict_time[model_name] = temps_exe  
        
    #Pour les Dataframe :    
    # Concaténation des métriques en un DataFrame et inversion des colonnes et index :
    df_metrics = pd.concat(dict_metric.values(), axis=1).T 
    #Enregistrement des prédictions dans un dataframe :
    df_prediction = pd.DataFrame(dict_prediction)
    
    return (df_metrics,df_prediction,dict_model_trained, dict_time)

def enregistrement_model(pretrained_models):
    """Prend en entrée un dictionnaire sous forme {nom_modèle : modèle préentrainé}"""
    for i in pretrained_models:
        model_name = i
        current_model = pretrained_models[model_name]
        model_filename = os.path.join(f"{model_name}.pkl")
        #Enregistrement des modèles préentrainé :
        with open(model_filename, 'wb') as model_file:
            pickle.dump(current_model, model_file)
        print(f'{colored(model_name,"blue")} enregistré sous {colored(model_filename,"green")}')

### Fonction pour la visualisation des résultat

In [ ]:
 #visualisation du temp d'entrainement
def graph_time(dictionnary_time):
    """Fonction qui récupère un dictionnaire contenant les informations sur la durée d'entrainement des modèles et renvoie un barplot"""
    df_time = pd.DataFrame.from_dict(dictionnary_time, orient = "index", columns=["Duree d'entrainement"])
    plt.figure(figsize=(16,4))
    plt.grid()
    ax = sns.barplot(x = df_time.index, y = df_time.columns[0], data = df_time)
    ax.set_xticklabels(labels = df_time.index, rotation = 45)
    plt.xlabel("Modèles")
    plt.title("Durée d'entrainement des différents modèles (en minutes)")
    plt.show()

    
#Visualisation des metrics avec un barplot
def metric_plot(df_metric):
    mesure_reset = df_metric.reset_index()
    # Ensuite, ON utilise melt avec le nouvel index comme id_vars
    metric_df_melt = mesure_reset.melt(id_vars="index", var_name="Model", value_name="Score")

    plt.figure(figsize=(16,6))
    plt.grid()
    sns.barplot(data=metric_df_melt, x="index", y="Score", hue="Model")
    plt.legend(bbox_to_anchor=(1,1))
    plt.title('Comparaison des scores des modèles')
    plt.show()
    
#visualisation de la matrice de confusion
def matrice(prediction):
    cf = metrics.confusion_matrix(y_true=y_test, y_pred=prediction)
    plt.figure(figsize=(4,4))
    ax = sns.heatmap(cf, annot = True, linewidths=0.8, linecolor="black", fmt = ".0f",cbar=False, cmap = "Blues")
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Valeurs réelles')
    plt.title("Confusion Matrix")
    plt.show()
    
#visualisation de la courbe ROC   
def ROC(model):
    """Fonction pour obtenir la courbe ROC pour la classification multiclasse"""
    n_classes = len(np.unique(y_test))
    # Binarisez les étiquettes (labels) pour pouvoir les utiliser dans roc_curve
    y_test_binarized = label_binarize(y_test, classes=model.classes_)
    predicted_probabilities = model.predict_proba(X_test.values)

    # Initialisez la figure pour le tracé
    plt.figure(figsize=(10, 8))

    # Calculez la courbe ROC pour chaque classe
    for i in range(len(model.classes_)):
        fpr, tpr, _ = metrics.roc_curve(y_test_binarized[:, i], predicted_probabilities[:, i])
        roc_auc = metrics.auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Classe {model.classes_[i]} (AUC = {roc_auc:.2f})')

    # Tracer la ligne en pointillés représentant la performance aléatoire
    plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Aléatoire')

    # Ajoutez des légendes, un titre et des étiquettes d'axe
    plt.legend(loc="lower right")
    plt.xlabel('Taux de faux positifs (FPR)')
    plt.ylabel('Taux de vrais positifs (TPR)')
    plt.title('Courbe ROC pour un problème de classification multiclasse')
    plt.grid(True)
    plt.show()

## 2. Entrainement et enregistrement des modèles


In [ ]:
##### Liste des modèles utilisés :
all_model = [
    DummyClassifier(strategy="most_frequent"),
    LogisticRegression(),
    KNeighborsClassifier(),
    SGDClassifier(loss="modified_huber"), 
    RandomForestClassifier(),
    AdaBoostClassifier(),
    XGBClassifier()
            ]

#Remarque : loss = "modified_huber" SGDC,cela permet d'utiliser le "predict_proba(X_test.values)"
# c'est indispensable à notre fonction ROC (car on utilise predict_proba et label_binarizer)
#sinon cela ne fonctionne pas car la courbe ROC n'est pas adapté aux problèmes non binaires

In [ ]:
#Entrainement en série :
mesure, prediction,model_entraine, exe_time = trainig_series(all_model)

In [ ]:
#Sauvegarde des modèles préentrainé :
enregistrement_model(model_entraine)

## Visualisation des premiers résultats :

In [ ]:
metric_plot(mesure)

In [ ]:
graph_time(exe_time)

In [ ]:
def matrice(pred, nrow, ncol):
    plt.figure(figsize=(14, 14))  
    for i, col in enumerate(pred, 1):
        cf = metrics.confusion_matrix(y_true=y_test, y_pred=pred[col])
        ax = plt.subplot(nrow, ncol, i)
        sns.heatmap(cf, annot=True, ax=ax,linewidths=0.8, linecolor="black",fmt=".0f", cbar=False, cmap="Blues")
        ax.set_xlabel('Prédictions')
        ax.set_ylabel('Valeurs réelles')
        ax.set_title(f"{col}")
    plt.tight_layout()  # Pour éviter les chevauchements
    plt.show()

In [ ]:
def ROC(model_list, nrow,ncol):
    """Fonction pour obtenir la courbe ROC pour la classification multiclasse"""
    n_classes = len(np.unique(y_test))
    plt.figure(figsize=(15, 10))  

    # Binarisation de la target pouvoir l'utiliser dans roc_curve
    y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))
    
    # Boucle sur chaque modèle et on trace la courbe ROC dans un sous-graphique
    for i, model in enumerate(model_list, 1):
        plt.subplot(nrow, ncol, i)  
        predicted_probabilities = model_list[model].predict_proba(X_test.values)
        
        # On calcul et on trace la courbe ROC pour chaque classe
        for j in range(n_classes):
            fpr, tpr, _ = metrics.roc_curve(y_test_binarized[:, j], predicted_probabilities[:, j])
            roc_auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'Classe {j} (AUC = {roc_auc:.2f})')
        
        # On trace la ligne en pointillés représentant la performance aléatoire
        plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Aléatoire')
        
        #legende :
        plt.legend(loc="lower right")
        plt.xlabel('Taux de faux positifs (FPR)')
        plt.ylabel('Taux de vrais positifs (TPR)')
        plt.title(f'Courbe ROC - Modèle {model}')
        plt.grid()
    
    plt.tight_layout()  
    plt.show()

In [ ]:
# Utilison la fonction ROC avec notre liste de modèles préentraînés
ROC(model_entraine,3,3)

# Selection du modèle 
- Au vu des performance, Randomforestclassifier et XGBClassifier offrent les meilleur résultats de base, donc nous allons en conserver un des deux

In [ ]:
#Recuperation du XGBC :
final_model = model_entraine['XGBClassifier']

# Prédiction sur de nouvelles données :
- Nous allons utiliser notre modèle sur les données test qui nous ont été fournies
- Pour cela, nous allons créé une fonction permettant de prétraiter nos données directement 

In [ ]:
df_test = pd.read_csv(path_test)
print(df_test.shape)
df_test.head()

In [ ]:
def new_prediction(model, data):
    #Récupération des mêmes features que celles utilisés pour l'entrainement des modèles
    columns = model.feature_names_in_
    #Récupération de l'id du jeu de données
    id_data = data["id"]
    #Préparation du dataframe à tester :
    X = data[columns]
    #Standardisation des données : 
    X[var_cont] = scaler.transform(X[var_cont])
    
    #Récupération des prédictions
    numeric_prediction = model.predict(X)
    
    #Conversion des prédiction en données d'origine (textuelles et non numérique)
    original_prediction = labelencoder.inverse_transform(numeric_prediction)
    
    #Transformation des prédictions en dataframe avec l'id en index
    prediction_df = pd.DataFrame(original_prediction, columns = ["Target"], index = id_data)
    return prediction_df


In [ ]:
submission_data = new_prediction(final_model, df_test)
submission_data.head()

In [ ]:
submission_data.to_csv("XGBClassifer_prediction.csv")

## Après soumission du modèle : Résultat 0.76239

# Amélioration par GridSearchCV

In [ ]:
def Grid(model, param):
    """Le modèle sera entrainé sur 5 splits
    et renverra les prédictions du meilleur modèles ainsi que son temps d'entrainement"""
    
 # Mesurer le temps de début
    start_time = time.time()
        
    #Entrainement du grid sur les paramètres
    grid = GridSearchCV(estimator=model, param_grid=param, cv=5,verbose=0,
    n_jobs=-1
)
    grid.fit(X_train, y_train)
    # Mesurer le temps de fin
    end_time = time.time()
    
    #Recuperation des meilleurs hyper parametres :
    best_model = grid.best_estimator_
    
    # Convertir X_test en un format compatible si nécessaire
    X_test_transformed = X_test.values  

    
    #Recuperation des predictions
    yp = best_model.predict(X_test_transformed)
    
    #Calcul temps d'execution:
    training_time = round(end_time- start_time  , 3)
    #On converti en minutes :
    training_time_min = round(training_time/60,2)
    # Récupération du nom du modèle et du chemin d'enregistrement
#     model_name = type(current_model).__name__
#     dict_time = {model_name:training_time_min}
    
    return yp, training_time_min, best_model


In [ ]:
# Définition des paramètres de la grille
param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres dans le modèle
     'max_depth': [3, 4, 5],  # Profondeur maximale de chaque arbre
     'learning_rate': [0.05, 0.1, 0.2],  # Taux d'apprentissage
     'min_child_weight': [1, 3, 5],  # Poids minimum des enfants (minimum sum of instance weight(hessian) needed in a child)
     'gamma': [0, 0.1, 0.2],  # Réduction de la perte minimale requise pour effectuer une partition sur une feuille
     'colsample_bytree': [0.8, 1.0],  # Proportion des colonnes utilisées pour entraîner chaque arbre
    "learning_rate": (0.05, 0.10, 0.15),
}
     

In [ ]:
pred_grid, time_grid, best_model_grid = Grid(XGBClassifier(), param=param_grid)

In [ ]:
df_metric_grid = metric(pred_grid, "XGBClassifier" )

In [ ]:
metric_plot(df_metric_grid)

In [ ]:
graph_time({type(best_model_grid).__name__:time_grid})

In [ ]:
best_model_grid.score(X_test,y_test)

In [ ]:
model_filename = "XGBClassifier.pkl"

In [ ]:
#Enregistrement des modèles préentrainé :
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model_grid, model_file)
    print(f'{colored("XGBClassifier","blue")} enregistré sous {colored(model_filename,"green")}')

In [ ]:
submission_data = new_prediction(best_model_grid, df_test)
submission_data.to_csv("XGBC_GRID_prediction.csv")

## Après soumission du modèle : Résultat 0.80248
### Nous avons légèrement amélioré le modèle